dev-01: 전처리한 데이터셋

dev-02: 전처리 안한 데이터셋

In [98]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_upstage import UpstageEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from dotenv import load_dotenv
import os
import re
load_dotenv()

def is_toc_or_appendix_page(text, page_num, total_pages):
    """목차나 부록 페이지인지 확인하는 함수"""
    # 앞부분 목차 페이지 체크 (처음 10페이지 내에서)
    if page_num <= 10:
        toc_patterns = [
            r'목\s*차',
            r'책의\s*차례',
            r'제\s*\d+\s*장',
            r'CONTENTS',
            r'\d+\.\s*.+\s*\.{2,}\s*\d+',  # 챕터 번호와 페이지 번호를 잇는 점선 패턴
        ]
        return any(re.search(pattern, text) for pattern in toc_patterns)
    
    # 뒷부분 부록 페이지 체크 (마지막 20페이지 내에서)
    if page_num >= total_pages - 20:
        appendix_patterns = [
            r'부\s*록',
            r'찾아보기',
            r'APPENDIX',
            r'INDEX',
            r'참고문헌',
            r'용어\s*찾기',
            r'찾아\s*보기',
        ]
        return any(re.search(pattern, text) for pattern in appendix_patterns)
    
    return False

def filter_sentences(text):
    """특정 패턴의 문장을 제거하는 함수"""
    sentences = re.split(r'[.!?]\s+', text)
    filtered_sentences = []
    
    for sentence in sentences:
        # 문장이 비어있지 않고
        # "?"로 끝나지 않고
        # "보자"로 끝나지 않는 경우만 포함
        if sentence.strip() and \
           not sentence.strip().endswith('?') and \
           not sentence.strip().endswith('보자'):
            filtered_sentences.append(sentence)
    
    return '. '.join(filtered_sentences)

def process_textbooks(folder_path, exclude_words, index_name):
    """
    PDF 교과서들을 처리하여 Pinecone에 저장하는 함수
    
    Args:
        folder_path (str): PDF 파일들이 있는 폴더 경로
        exclude_words (list): 제외할 단어 리스트
        index_name (str): Pinecone 인덱스 이름
    """
    # Pinecone 초기화
    embeddings_passage = UpstageEmbeddings(model="embedding-passage") #4096
    docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings_passage)
    
    # 모든 문서를 저장할 리스트
    all_documents = []
    
    # PDF 파일 처리
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            # 파일명에서 과목 추출 (_로 분리된 3번째 단어)
            subject = filename.split('_')[2]
            print(subject)
            # PDF 로드
            loader = PyMuPDFLoader(os.path.join(folder_path, filename))
            pages = loader.load()
            total_pages = len(pages)

            # 페이지 필터링 및 문장 처리
            filtered_pages = []
            print(f"Processing {filename}: {total_pages} pages")
            
            for page_num, page in enumerate(pages, 1):
                
                # 목차/부록 페이지 건너뛰기
                if is_toc_or_appendix_page(page.page_content, page_num, total_pages):
                    continue
                
                # 제외 단어 체크
                should_exclude = False
                for word in exclude_words:
                    if word in page.page_content:
                        should_exclude = True
                        break
                
                if not should_exclude:
                    # 특정 패턴의 문장 필터링
                    filtered_content = filter_sentences(page.page_content)
                    
                    # 길이가 너무 짧은 페이지 건너뛰기
                    if len(filtered_content) < 50:
                        continue
                    
                    # 필터링된 내용이 있는 경우만 추가
                    if filtered_content.strip():
                        page.page_content = filtered_content
                        page.metadata['subject'] = subject
                        
                        # file 이름만 가져오기"/"로 구분
                        source = filename.split("/")[-1]
                        
                        page.metadata = {
                            'subject': subject,
                            'page_num': page_num,
                            'total_pages': total_pages,
                            'source': source,
                        }
                        
                        filtered_pages.append(page)
            
            print(f"Filtered pages: {len(filtered_pages)}")
            all_documents.extend(filtered_pages)
    
    # RecursiveCharacterTextSplitter로 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=150,
    )
    
    chunks = text_splitter.split_documents(all_documents)
    
    # Pinecone에 저장
    docsearch.add_documents(chunks)
    
    return len(chunks)

folder_path = "data"
exclude_words = ["창의·융합", "창의·융복합", "마무리", "자료 해석", "부록", "정답과 해설"]

num_chunks = process_textbooks(
    folder_path=folder_path,
    exclude_words=exclude_words,
    index_name="dev-01"
)

print(f"총 {num_chunks}개의 청크가 Pinecone에 저장되었습니다.")

생명과학Ⅱ
Processing 천재교육_고등교과서_생명과학Ⅱ_이준규(15개정)_교과서 본문.pdf: 248 pages
Filtered pages: 201
물리학Ⅱ
Processing 천재교육_고등교과서_물리학Ⅱ_강남화(15개정)_교과서 본문.pdf: 224 pages
Filtered pages: 161
화학Ⅱ
Processing 천재교육_고등교과서_화학Ⅱ_노태희(15개정)_교과서 본문.pdf: 232 pages
Filtered pages: 177
지구과학Ⅱ


/opt/homebrew/anaconda3/envs/venv/lib/python3.12/site-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 225 of document data/천재교육_고등교과서_지구과학Ⅱ_오필석(15개정)_교과서 본문.pdf
  warnings.warn(


Processing 천재교육_고등교과서_지구과학Ⅱ_오필석(15개정)_교과서 본문.pdf: 248 pages
Filtered pages: 159
정보처리기사
Processing 2025_기출문제집_정보처리기사_필기_핵심요약.pdf: 116 pages
Filtered pages: 112
정치와법
Processing 천재교과서_고등교과서_정치와법_김왕근(15개정)_교과서 본문.pdf: 224 pages
Filtered pages: 199
총 3060개의 청크가 Pinecone에 저장되었습니다.


In [99]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_upstage import UpstageEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from dotenv import load_dotenv
import os
import re
load_dotenv()

def process_textbooks(folder_path, exclude_words, index_name):
    """
    PDF 교과서들을 처리하여 Pinecone에 저장하는 함수
    
    Args:
        folder_path (str): PDF 파일들이 있는 폴더 경로
        exclude_words (list): 제외할 단어 리스트
        index_name (str): Pinecone 인덱스 이름
    """
    # Pinecone 초기화
    embeddings_passage = UpstageEmbeddings(model="embedding-passage") #4096
    docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings_passage)
    
    # 모든 문서를 저장할 리스트
    all_documents = []
    
    # PDF 파일 처리
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            # 파일명에서 과목 추출 (_로 분리된 3번째 단어)
            subject = filename.split('_')[2]
            print(subject)
            # PDF 로드
            loader = PyMuPDFLoader(os.path.join(folder_path, filename))
            pages = loader.load()
            total_pages = len(pages)

            # 페이지 필터링 및 문장 처리
            filtered_pages = []
            print(f"Processing {filename}: {total_pages} pages")
            
            for page_num, page in enumerate(pages, 1):
                
                
                
                
                # 특정 패턴의 문장 필터링
                filtered_content = page.page_content
                
                # 길이가 너무 짧은 페이지 건너뛰기
                if len(filtered_content) < 50:
                    continue
                
                # 필터링된 내용이 있는 경우만 추가
                if filtered_content.strip():
                    page.page_content = filtered_content
                    page.metadata['subject'] = subject
                    
                    # file 이름만 가져오기"/"로 구분
                    source = filename.split("/")[-1]
                    
                    page.metadata = {
                        'subject': subject,
                        'page_num': page_num,
                        'total_pages': total_pages,
                        'source': source,
                    }
                    
                    filtered_pages.append(page)
            
            print(f"Filtered pages: {len(filtered_pages)}")
            all_documents.extend(filtered_pages)
    
    # RecursiveCharacterTextSplitter로 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=150,
    )
    
    chunks = text_splitter.split_documents(all_documents)
    
    # Pinecone에 저장
    docsearch.add_documents(chunks)
    
    return len(chunks)

folder_path = "data"
exclude_words = ["창의·융합", "창의·융복합", "마무리", "자료 해석", "부록", "정답과 해설"]

num_chunks = process_textbooks(
    folder_path=folder_path,
    exclude_words=exclude_words,
    index_name="dev-02"
)

print(f"총 {num_chunks}개의 청크가 Pinecone에 저장되었습니다.")

생명과학Ⅱ
Processing 천재교육_고등교과서_생명과학Ⅱ_이준규(15개정)_교과서 본문.pdf: 248 pages
Filtered pages: 244
물리학Ⅱ
Processing 천재교육_고등교과서_물리학Ⅱ_강남화(15개정)_교과서 본문.pdf: 224 pages
Filtered pages: 221
화학Ⅱ
Processing 천재교육_고등교과서_화학Ⅱ_노태희(15개정)_교과서 본문.pdf: 232 pages
Filtered pages: 231
지구과학Ⅱ
Processing 천재교육_고등교과서_지구과학Ⅱ_오필석(15개정)_교과서 본문.pdf: 248 pages
Filtered pages: 245
정보처리기사
Processing 2025_기출문제집_정보처리기사_필기_핵심요약.pdf: 116 pages
Filtered pages: 115
정치와법
Processing 천재교과서_고등교과서_정치와법_김왕근(15개정)_교과서 본문.pdf: 224 pages
Filtered pages: 222
총 4205개의 청크가 Pinecone에 저장되었습니다.
